In [1]:
from kaggle_environments import make
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.game_objects import Unit
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math, sys
import numpy as np
import random
from IPython.display import clear_output 
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_hub as hub
from collections import deque
import random
import math

Loading environment football failed: No module named 'gfootball'


In [2]:
# this is the basic agent definition. At the moment this agent does nothing (and actually will survive for a bit before getting consumed by darkness)
def agent(observation, configuration):
    global game_state
    
    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
#         print("new episode")
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
#     print(observation['reward'])

    return actions

In [3]:
# run another match but with our empty agent
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2}, debug=True)
steps = env.run([agent, agent])

In [4]:
def get_inputs(game_state):
    # Teh shape of the map
    w,h = game_state.map.width, game_state.map.height
    # The map of ressources
    M = [ [0  if game_state.map.map[j][i].resource==None else game_state.map.map[j][i].resource.amount for i in range(w)]  for j in range(h)]
    
    M = np.array(M).reshape((h,w,1))
    
    # The map of units features
    U = [ [[0,0,0,0,0] for i in range(w)]  for j in range(h)]
    units = game_state.players[0].units
    for i in units:
        U[i.pos.y][i.pos.x] = [i.type,i.cooldown,i.cargo.wood,i.cargo.coal,i.cargo.uranium]

    U = np.array(U)
    
    # The map of cities featrues
    e = game_state.players[1].cities
    C = [ [[0,0,0,0] for i in range(w)]  for j in range(h)]
    for k in e:
        citytiles = e[k].citytiles
        for i in citytiles:
            C[i.pos.y][i.pos.x] = [i.cooldown,e[k].fuel,e[k].light_upkeep,e[k].team]

    C = np.array(C)
#     print(M.shape,U.shape,C.shape)
    # stacking all in one array
    E = np.dstack([M,U,C])
    return E

input_shape = get_inputs(game_state).shape
# The shape of input
input_shape 

(32, 32, 10)

In [5]:
def get_model(game_state):
    inputs = keras.Input(shape=get_inputs(game_state).shape,name = 'The game map')
    
    # Try to play with the next layers in order to enhance the brean of your agent
    c = layers.Conv2D(8,(1,1),activation = "relu")(inputs)
    c = layers.Conv2D(8,(1,1),activation = "relu")(c)
    c = layers.Conv2D(8,(1,1),activation = "relu")(c)
    
    # The next layer will define the direction among the 5 options, because we can take only one direction we had to use softmax activation
    output1 = layers.Dense(5,activation = "softmax",name ='direction')(c)
    
    # The next layer will define the option token by the unit (move, build,...), because one unit can take only one option we use sotmax activation
    output2 = layers.Dense(3,activation = "softmax",name="option" )(c)
    output = layers.concatenate([output1,output2])
    model = keras.Model(inputs = inputs, outputs = output)
    model.compile(loss='mse', optimizer="adam")
    return model

model =get_model(game_state)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
The game map (InputLayer)       [(None, 32, 32, 10)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 8)    88          The game map[0][0]               
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 8)    72          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 8)    72          conv2d_1[0][0]                   
______________________________________________________________________________________________

In [6]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=1,
    show_dtype=1,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [7]:
x = get_inputs(game_state)
y = model.predict(np.asarray([x]))[0]
y.shape

(32, 32, 8)

In [8]:
def get_prediction_actions(y,units):
    # move
    mv = np.argmax(y[:,:,:5],axis = 2) # the index in this list  [c s n w e]
    
    choice = np.argmax(y[:,:,5:],axis = 2)
    actions = []
    for i in units:
        d = "csnwe"[mv[i.pos.y,i.pos.x]]
        if choice[i.pos.y,i.pos.x]==0 and i.can_act():actions.append(i.move(d))
        elif choice[i.pos.y,i.pos.x]==1 and i.can_build(game_state.map):actions.append(i.build_city())
        elif choice[i.pos.y,i.pos.x]==2:actions.append(i.pillage())
        
    return actions,y[:,:,5:]
get_prediction_actions(y,game_state.players[0].units)[0]

['m u_1 c']

In [9]:
Last_State = {}
learning_rate = 0.01
gamma = 0.95
epsilon = 1.0
epsilon_final = 0.01
epsilon_decay = 0.995
# we declare this global game_state object so that state persists across turns so we do not need to reinitialize it all the time
game_state = None
model = None
last_reward = 0
def agent(observation, configuration):
    global game_state,epsilon,model,last_reward
    
    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
        # Inistialise the model
        if not model:
            model= get_model(game_state)
#             model.load_weights('../input/keras-lux-ai-weights/model.h5')
        
    else:
        game_state._update(observation["updates"])
    

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
#     print(width, height)

    # Get Prediction of actions
    x = get_inputs(game_state)
#     print(x.shape)
    y = model.predict(np.asarray([x]))[0]
    
    if random.random()<epsilon:
        y = np.random.rand(*y.shape)
    print("eps ",epsilon,end= " | ") 
    actions,_ = get_prediction_actions(y,player.units)
    
    print("Reward",observation["reward"])
    # Model Learing
    
    if observation.player in Last_State:
        _x,_y,_ = Last_State[observation.player]
        state,next_state,reward = _x,x,observation["reward"]
#         print(_y.shape,_.shape)
#         reward = sigmoid(reward)
        if reward > last_reward:r=1
        elif reward < last_reward:r = -1
        else:r = 0.5
            
        r = r+ gamma * np.amax(_y,axis = 2)
        # print(reward.shape)

        for i in player.units:
            print(_y[i.pos.x,i.pos.y].shape,_[i.pos.x,i.pos.y].shape)
            _y[i.pos.y,i.pos.x][[0,1,2,3,4]] = r[i.pos.x,i.pos.y]
            _y[i.pos.y,i.pos.x][5] = r[i.pos.y,i.pos.x] if r[i.pos.y,i.pos.x]>0.5 else 1-r[i.pos.y,i.pos.x]
            _y[i.pos.y,i.pos.x][6] = r[i.pos.y,i.pos.x] if r[i.pos.y,i.pos.x]>0.5 else 1-r[i.pos.y,i.pos.x]
            _y[i.pos.y,i.pos.x][7] = r[i.pos.y,i.pos.x] if r[i.pos.y,i.pos.x]>0.5 else 1-r[i.pos.y,i.pos.x]
        
        _y = y + learning_rate*_y
            
        model.fit(np.asarray([state]),np.asarray([_y]), epochs=1, verbose=1)
        if epsilon > epsilon_final:
            epsilon*= epsilon_decay
    Last_State[observation.player] = [x,y,_]
    last_reward = observation["reward"]
    return actions

In [10]:
episodes = 10

for eps in range(episodes):
    clear_output()
    print("=== Episode {} ===".format(eps))
    env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2}, debug=True)
    steps = env.run(["simple_agent", agent])

=== Episode 9 ===
eps  0.00998645168764533 | Reward 0
(8,) (3,)
1/1 [==============================] - 0s 9ms/step - loss: 1.4558e-04
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 17ms/step - loss: 2.8388e-05
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 7ms/step - loss: 7.3613e-05
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 21ms/step - loss: 2.0478e-05
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 5ms/step - loss: 2.0940e-05
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 5ms/step - loss: 2.1281e-05
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 8ms/step - loss: 2.0649e-05
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 6ms/step - loss: 2.0379e-05
eps  0.00998645168764533

eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 4ms/step - loss: 2.0488e-05
[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 48; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 4ms/step - loss: 2.0446e-05
[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 49; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 4ms/step - loss: 2.0409e-05
[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 50; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 4ms/step - loss: 2.0376e-05
[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 51; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 4ms/step - loss: 2.0346e-05
[WARN] (match_9Uoar29PBvRD) - Ag

[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 85; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 3ms/step - loss: 2.0266e-05
[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 86; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 3ms/step - loss: 2.0272e-05
[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 87; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 5ms/step - loss: 2.0278e-05
[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 88; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [==============================] - 0s 3ms/step - loss: 2.0285e-05
[WARN] (match_9Uoar29PBvRD) - Agent 1 tried to move unit u_2 off map; turn 89; cmd: m u_2 s
eps  0.00998645168764533 | Reward 10001
(8,) (3,)
1/1 [=====

In [11]:
model.save_weights("model.h5")

In [15]:
from pathlib import Path
p = Path('/kaggle_simulations/agent/')
if p.exists():
    sys.path.append(str(p))
else:
    p = Path('__file__').resolve().parent

In [16]:
def agent(observation, configuration):
    global game_state,epsilon,model
    
    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
        print("Creating model..")
        model =get_model(game_state)
        print("Load model weight..")
        try:
            model.load_weights( str(p/'model.h5'),  by_name=True, skip_mismatch=True)
        except Exception as e:
            print('Error in model load')
            print(e)
#         model = tf.keras.models.load_model('model.h5')
        print("Done crating mdoel")
        
        
    else:
        game_state._update(observation["updates"])
    

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    # Get Prediction of actions
    x = get_inputs(game_state)
    y = model.predict(np.asarray([x]))[0]
    actions,_ = get_prediction_actions(y,player.units)
    return actions

In [13]:
env = make("lux_ai_2021", configuration={"seed": 56221, "loglevel": 2}, debug=True)

In [19]:
steps = env.run([agent, agent])

Creating model..
Load model weight..
Done crating mdoel
Creating model..
Load model weight..
Done crating mdoel
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 1; cmd: p u_2
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 3; cmd: p u_2
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 5; cmd: p u_2
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 7; cmd: p u_2
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 9; cmd: p u_2
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 11; cmd: p u_2
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 13; cmd: p u_2
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 15; cmd: p u_2
[WARN] (match_sypKLAxODT8K) - Agent 1 tried to pillage tile with cooldown: 1; turn 17; cmd: p u_2
[WARN] (mat

In [20]:
env.render(mode="ipython", width=600, height=800)